<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/09-CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross Validation

En esta notebook practicaremos el uso de la validación cruzada. La validación cruzada es una técnica fundamental en machine learning para evaluar el rendimiento de un modelo de manera robusta y evitar el overfitting. En lugar de dividir los datos una sola vez en conjuntos de entrenamiento y prueba (como en el train-test split), la validación cruzada repite este proceso múltiples veces, dividiendo los datos en *k* particiones (folds) y rotando cuál se usa para validación. Esto permite:

* Estimar mejor la generalización del modelo, calculando métricas promedio sobre todas las iteraciones.
* Reducir la dependencia de una división aleatoria específica, especialmente útil en datasets pequeños.
* Optimizar hiperparámetros (por ejemplo, GridSearch**CV**), garantizando que la configuración elegida sea más estable.


<img src="https://drive.google.com/uc?id=19vqfNoSDSmG4TygQRoSE-WGceThEtgWm" alt="alt text" width="500">


Usaremos la implementación [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score).

## Ejemplo de uso

In [ ]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X, y = data.data, data.target

print(X.shape)
print(y.shape)

In [ ]:
data.feature_names

Veamos la descripción del dataset

In [ ]:
print(data.DESCR)

Como podemos ver, es un dataset ligeramente desvalanceado. No hay umbrales precisos. Una guía empírica es:

* Balanceado: La clase minoritaria representa > 30% del total.
* Desbalanceado moderado: Clase minoritaria entre 10% y 30%.
* Extremadamente desbalanceado: Clase minoritaria < 10%.
* Si la clase minoritaria tiene < 5% de los datos, se considera un problema severo (requiere técnicas especiales como oversampling/SMOTE o cost-sensitive learning).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ratio = np.unique(y,return_counts=True)[1][0]/len(y)

labels, countings = np.unique(y,return_counts=True)

plt.figure()
plt.bar(labels,countings)
plt.xticks(labels)
plt.title(f"Distribución de clases\n Ratio:{np.round(ratio,2)}")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, # IMPORTANTE!
                                                    random_state=842
                                                    )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Suponiendo que y_train y y_test son tus datos
train_labels, train_counts = np.unique(y_train, return_counts=True)
test_labels, test_counts = np.unique(y_test, return_counts=True)

plt.figure(figsize=(9, 4))

# Subplot 1: Train
plt.subplot(1, 2, 1)
plt.bar(train_labels, train_counts, color='blue', label='Train')
plt.xticks(train_labels)
plt.title("Distribución de clases - Train")
plt.xlabel("Clases")
plt.ylabel("Conteo")

# Subplot 2: Test
plt.subplot(1, 2, 2)
plt.bar(test_labels, test_counts, color='orange', label='Test')
plt.xticks(test_labels)
plt.title("Distribución de clases - Test")
plt.xlabel("Clases")
plt.ylabel("Conteo")

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")

print(f"Train F1 Score: {f1_score(y_train,y_train_pred)}")
print(f"Test F1 Score: {f1_score(y_test, y_test_pred)}")

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Scores: {scores}")
print(f"Promedio: {np.mean(scores)}")

In [ ]:
model.get_depth()

Modifiquemos el modelo:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")

print(f"Train F1 Score: {f1_score(y_train,y_train_pred)}")
print(f"Test F1 Score: {f1_score(y_test, y_test_pred)}")

scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Scores: {scores}")
print(f"Promedio: {np.mean(scores)}")

**Como podemos ver, el cross validation exhibe el problema de overfitting.**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

model = DecisionTreeClassifier(max_depth=None)
model.fit(X, y)
y_pred = model.predict(X)

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Train F1 Score: {f1_score(y_train,y_train_pred)}")

scores = cross_val_score(model, X, y, cv=5)
print(f"Accuracy Promedio CV: {np.mean(scores)}")

🔵 Observar cómo podemos hacer un entrenamiento con todo el conjunto de datos y tener una buena estimación del rendimiento del modelo, aún sin tener un conjunto de prueba.

## Cross Validation y Overfitting

En este experimento compararemos el rendimiento de dos modelos de regresión (uno simple, lineal, y otro complejo, polinómico de grado alto) para exhibir el fenómeno de *overfitting* y cómo la validación cruzada ayuda a detectarlo.

**Lo mostraremos usando un problema de regresión**, para hacer enfasis que no es una herramienta exclusiva de la clasificación.

Generamos datos sintéticos basados en una función seno con ruido, y se evalúa:

* Train-Test Split: Midiendo el MAPE (Error Porcentual Absoluto Medio) en entrenamiento y prueba, donde el modelo complejo suele tener bajo error en entrenamiento pero alto en prueba (señal de overfitting).

* 5-Fold Cross-Validation: Calculando el MAPE promedio en múltiples particiones, revelando que el modelo complejo tiene mayor variabilidad y peor generalización.

* Visualización: Gráficos muestran cómo el modelo complejo sigue el ruido (izquierda) y su alta dispersión de errores en validación cruzada (derecha).

**La validación cruzada proporciona una evaluación más confiable que una sola división train-test, exponiendo la inestabilidad de modelos sobreajustados.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_percentage_error, make_scorer


# Generamos datos sintéticos con ruido
np.random.seed(42)
X = np.linspace(0, 1, 30)
y = np.sin(2 * np.pi * X) + np.random.normal(0, 0.2, 30)  # Función seno + ruido

plt.figure(figsize=(8, 5))
plt.scatter(X, y)
plt.show()

In [ ]:
grado = 12

X = X.reshape(-1, 1)  # Formato correcto para estimadores de scikit-learn

# Dividir en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir modelos
modelo_simple = LinearRegression()
modelo_complejo = make_pipeline(
    PolynomialFeatures(degree=grado),  # Polinomio de grado alto (overfitting)
    LinearRegression()
)

# Entrenar y evaluar con train-test split
modelo_simple.fit(X_train, y_train)
modelo_complejo.fit(X_train, y_train)

# Predecir
y_pred_simple = modelo_simple.predict(X_test)
y_pred_complejo = modelo_complejo.predict(X_test)

y_pred_simple_train = modelo_simple.predict(X_train)
y_pred_complejo_train = modelo_complejo.predict(X_train)

# Calcular errores
mse_simple = mean_absolute_percentage_error(y_test, y_pred_simple)
mse_complejo = mean_absolute_percentage_error(y_test, y_pred_complejo)

print(f"MAPE (Simple - Train): {mean_absolute_percentage_error(y_train, y_pred_simple_train):.2f}")
print(f"MAPE (Simple - Test): {mse_simple:.2f}",end="\n"+50*"-"+"\n")

print(f"MAPE (Complejo - Train): {mean_absolute_percentage_error(y_train, y_pred_complejo_train):.2f}")
print(f"MAPE (Complejo - Test): {mse_complejo:.2f}")

In [ ]:
# Definir la métrica MAPE como scorer personalizado
mape_scorer = make_scorer(
    mean_absolute_percentage_error,
    greater_is_better=False  # Porque el MAPE es un error (menor es mejor)
)

# Evaluar con Cross-Validation (5-Fold)
cv_scores_simple = cross_val_score(modelo_simple, X, y, cv=5, scoring=mape_scorer)
cv_scores_complejo = cross_val_score(modelo_complejo, X, y, cv=5, scoring=mape_scorer)

mape_simple = -cv_scores_simple.mean()
mape_complejo = -cv_scores_complejo.mean()

print(f"MAPE Promedio CV (Simple): {mape_simple:.2f}")
print(f"MAPE Promedio CV (Complejo): {mape_complejo:.2f}")

In [ ]:
plt.figure(figsize=(12, 6))

# Gráfico de los datos y modelos
plt.subplot(1, 2, 1)
plt.scatter(X, y, s=20, label="Datos reales")
plt.plot(X, modelo_simple.predict(X), color='red', label="Modelo simple (grado 1)")
plt.plot(X, modelo_complejo.predict(X), color='green', label="Modelo complejo (grado 15)")
plt.title("Comparación de modelos")
plt.legend()

# Gráfico de errores en CV
plt.subplot(1, 2, 2)
plt.boxplot(
    [-cv_scores_simple, -cv_scores_complejo],
    tick_labels=["Modelo Simple", "Modelo Complejo"]
)
plt.title("Distribución del MAPE  en 5-Fold CV")
plt.ylabel("MAPE")

plt.tight_layout()
plt.show()